## The Question
Large Language Models perform really well on natural language generation tasks, sometimes equally or surpassing human abilities. Since GPT, natural language processing has seen many advancements, and ChatGPT showcased the power of LLMs on variety of tasks like question answering, generating long texts, and even simple logical reasoning. But, the answers were based out of the parametric memory of the model which was frozen, i.e., it could not update or fetch relevant data. Many a times the LLMs would invent results and output them: they would *hallucinate*. **Is there a way to make large language models fetch relevant information and generate accurate answers?**

## The Dataset
We are utilizing 3 chapters (chapter 1, 2, and 7) of the course textbook 'Artificial Intelligence: A Modern Approach' and a couple of slides from Prof. Keogh and Prof. LePendu as our additional context.

## The Method
To achieve our objective of eliminating hallucination and boost relevancy, we utilize Llama-Index, LangChain, and Ollama to build a RAG pipeline. Llama-Index is used to index the documents and store them in a vector database. LangChain serves the local Ollama model of Llama 2 7B. The chapters from the textbook and the slides are index and stored in a vector database which are then retrieved based on the similarity with the query passed to the language model. We have used OpenAI's state-of-the-art text-embedding-ada-002 embedding model through their API. 

## Our hypothesis
We feel that supplementing the LLM with additional context can boost the relevance of the answer and stop it from hallucinating. The evaluation is done using the RAGAS approach where we measure how relevant and consistent the generated result is to the context and the query.   

## Why bother?
As the use of LLM is rising, one LLM cannot know everything in this universe. It is important to have specific LLM for a specific purpose. It is also important to have fresh data for the LLM to access and output relevant and factually correct results. The approach of RAG seems promising to alleviate most of the problems related to LLMs generation and correct usage can boost the utilization of the language models in various fields. 






# Main Jupyter Notebook for the CS205 Final Project
*This only works if run locally after completing all the installations mentioned in the README file of the project*

1. In this project we will explore a usecase of Large Language Models
2. We will start with how to use an LLM through HuggingFace, and explain some of the basic concepts behind an LLM. 
3. Once we have a good understading of how to use an LLM for generating text, we will explore Retrieval Augmented Generation (RAG). 

 For this project we have used Llama 2 7 Billion paramter model with OpenAI's text-embed-002 embedding model. Llama 2 7B was served locally by Ollama. We have used Llama Index and LangChain to interact with the LLM

#### The data store is at the root of the project directory with the name 'data'. Create a data repository before running the indexing and query cells

#### Let's understand how to use an LLM using HuggingFace

In [24]:
# attach to the same event-loop
import nest_asyncio

nest_asyncio.apply()

import os

In [25]:
import torch

#Import HuggingFace Transformer
from transformers import AutoModelForCausalLM, AutoTokenizer

/Users/kiranhk/anaconda3/envs/pyml-book/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [26]:
#Fetch Meta's OPT LLM with 1.3 billion parameters. This is quite a small model compared to the SOTA like GPT4V, etc.

model = AutoModelForCausalLM.from_pretrained('facebook/opt-1.3b')
tokenizer = AutoTokenizer.from_pretrained('facebook/opt-1.3b')

##### Open Pre-trained Transformer (OPT) is a collection of decoder-only transformer developed by Meta. 

In [27]:
input_text = 'I like CS205 Artificial Intelligence course, because'
tok_input = tokenizer(input_text, return_tensors='pt', add_special_tokens=True, truncation=True) #Create tokens from the given input and return PyTorch tensors

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


In [28]:
torch.manual_seed(123)

generated_output = model.generate(**tok_input, 
                                  max_new_tokens=200, 
                                  return_dict_in_generate=True, 
                                  do_sample=True) #Generation is deterministic. 
                                                  #To use top-k sampling, set do_sample=True to get different responses in each generation
                                                  #Set do_sample=False to have a deterministic generation each time


In [29]:
decoded_output = tokenizer.batch_decode(generated_output.sequences, skip_special_tokens=True)[0]
print(decoded_output)

I like CS205 Artificial Intelligence course, because of the breadth of material covered and the fact that it is an introductory course, so the material is "fresh". I think its a good course. (Yes, I do agree that I like the math, and i do agree that in all CS courses, the first course or two are usually the hardest ones).
Do you think you would still like it had you not taken a calculus course with higher math content


#### We saw text generation in the previous subsection. Now lets explore text summarization, a critical usecase of LLM

#### What is an embedding?

Embedding is a numerical representation of the text. The words in the vocabulary are mapped to a set of integers. These integers are then converted into another mathematical representation. This 'embedding' vector is of shape (n_tokens x embedding_dimension)

In [30]:
sentence = "Life is sweet and sugary, but eating ice cream on Everest is long"

In [31]:
tokenize1 = {s: i for i, s in enumerate(sorted(sentence.replace(',', '').split()))}

In [32]:
sentence_vec = torch.tensor([tokenize1[s] for s in sentence.replace(',', '').split()])
print(sentence_vec)

tensor([ 1,  8, 12,  2, 11,  3,  5,  6,  4, 10,  0,  8,  9])


The sentence1 is tokenized as shown in the output above. The words are mapped to an integer. 

In [33]:
torch.manual_seed(123)

In [34]:
embed = torch.nn.Embedding(len(sentence_vec), 10)
embedded_sentence = embed(sentence_vec).detach()

The sentence "Life is sweet and sugary, but eating ice cream in Manali is long" now has a representation like shown below. It is converted from text to a form which the machine understands. This embedding is the input to a language model.

In [35]:
print(embedded_sentence.shape)
print(embedded_sentence)

torch.Size([13, 10])
tensor([[ 0.6984, -1.4097,  0.1794,  1.8951,  0.4954,  0.2692, -0.0770, -1.0205,
         -0.1690,  0.9178],
        [ 0.2553, -0.5496,  1.0042,  0.8272, -0.3948,  0.4892, -0.2168, -1.7472,
         -1.6025, -1.0764],
        [-1.4205, -0.2238, -0.2548,  1.1517, -0.0179,  0.4264, -0.7657, -0.0545,
         -0.7321,  1.2347],
        [ 1.5810,  1.3010,  1.2753, -0.2010,  0.4965, -1.5723,  0.9666, -1.1481,
         -1.1589,  0.3255],
        [-0.9896,  0.7016, -0.9405, -0.4681, -0.8016, -0.8183, -1.1820, -0.2877,
         -0.6043,  0.6002],
        [-0.6315, -2.8400, -1.3250,  0.1784, -2.1338,  1.0524, -0.3885, -0.9343,
         -0.4991, -1.0867],
        [-1.4779,  1.1331, -1.2203,  1.3139,  1.0533,  0.1388,  2.2473, -0.8036,
         -0.2808,  0.7697],
        [-0.6596, -0.7979,  0.1838,  0.2293,  0.5146,  0.9938, -0.2587, -1.0826,
         -0.0444,  1.6236],
        [ 0.8805,  1.5542,  0.6266, -0.1755,  0.0983, -0.0935,  0.2662, -0.5850,
          0.8768,  1.6221]

In [36]:
sentence1 = "I feel like driving car today"
sentence2 = "I think I want to drive today"

In [37]:
from sentence_transformers import SentenceTransformer, util

model = SentenceTransformer('paraphrase-MiniLM-L6-v2')

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
1_Pooling/config.json: 100%|██████████| 190/190 [00:00<00:00, 1.57MB/s]
README.md: 100%|██████████| 3.69k/3.69k [00:00<00:00, 27.2MB/s]
config_sentence_transformers.json: 100%|██████████| 122/122 [00:00<00:00, 726kB/s]
pytorch_model.bin: 100%|██████████| 90.9M/90.9M [00:07<00:00, 12.5MB/s]
sentence_bert_config.json: 100%|██████████| 53.0/53.0 [00:00<00:00, 159kB/s]
special_tokens_map.json: 100%|██████████| 112/112 [00:00<00:00, 262kB/s]
tokenizer.json: 100%|██████████| 466k/466k [00:00<00:00, 3.58MB/s]
tokenizer_config.json: 100%|██████████| 314/314 [00:00<00:00, 1.09MB/s]
vocab.txt: 100%|██████████| 232k/232k [00:00<00:00, 13.0MB/s]
modules.json: 100%|██████████| 229

In [38]:
embedding1 = model.encode(sentence1)
embedding2 = model.encode(sentence2)

Evaluating a cosine similarity of 2 embeddings tells us how similar the vectors are semantically. This is particularly useful in our application where the query is matched with documents using a similarity function. 

In [39]:
util.pytorch_cos_sim(embedding1, embedding2)

tensor([[0.7208]])

#### What is RAG?

#### Retrieval Augmented Generation 

It is a technique in generative AI to boost the knowledge of an LLM. The LLM parameters are learned and not updated to the current information, so a specialized database of knowledge (could be private) is created for the LLM is access. This is a non-parametric memory, i.e, this information is not stored in the learned paramaters of the LLM. RAG combines retrieval with generation for content generation tasks.

How does an RAG work?

1. Retrieval: The model retrieves a set of top-k relevant documents that act as additional context for the query. Since the documents are stored in the database in the form of embeddings, the model can perform similarity searches in retrieval

2. Generation: Once the documents are retrieved, this serves as additional context along with the original input. The generative model can now use both the original input and retrieved context to generated the content. 

#### Implementing RAG with Llama Index using ChromaDB as the vector database. Ollama is used to serve Llama 2 locally

The data can be accessed at this link. Add this folder to the root of the project directory

https://drive.google.com/drive/folders/10wzRErO4Zlj6L3bLSqh9QtTLDkaUOuxS?usp=drive_link

In [40]:
import openai

from llama_index import VectorStoreIndex, SimpleDirectoryReader, ServiceContext, download_loader
from llama_index.vector_stores import ChromaVectorStore
from llama_index.storage.storage_context import StorageContext
from llama_index.embeddings import OpenAIEmbedding

from langchain.llms import Ollama

import chromadb

In [41]:
from dotenv import dotenv_values
from pathlib import Path

In [42]:
api_key = dotenv_values('../.env')["OPENAI_API_KEY"]
openai.api_key = api_key

os.environ["OPENAI_API_KEY"] = api_key # For RAGAS evaluation

In [43]:
#Set the embedding

llm = Ollama(model="llama2")
#embed_model = OllamaEmbeddings(base_url="http://localhost:11434", model="llama2") #Local Llama 2 embedding model
embed_model = OpenAIEmbedding() #Using OpenAI's text-embed-002

text-embedding-ada-002 is a powerful embedding model released by OpenAI. Like we discussed, an embedding represents a text mathematically in n-dimensions, and the distance between the embeddings can measure the similarity between the sentences or words. 

In [44]:
COLLECTION = "aiprof"
SLIDE_COLLECTION = 'slides'
PATH = '../chroma'

In [45]:
# create client and a new collection
db = chromadb.PersistentClient(path=PATH)
chroma_collection = db.get_or_create_collection(COLLECTION)

In [46]:
vector_store = ChromaVectorStore(chroma_collection=chroma_collection)
storage_context = StorageContext.from_defaults(vector_store=vector_store)
service_context = ServiceContext.from_defaults(llm=llm, embed_model=embed_model)

In [47]:
# load documents
documents = SimpleDirectoryReader("../data/AIMA/").load_data()

In [48]:
slides_reader = download_loader("PptxReader")
loader = slides_reader()
slides = []

for file in os.listdir("../data/slides/"):
    slides += loader.load_data("../data/slides/" + file)

In [49]:
information = documents + slides

In [50]:
index = VectorStoreIndex.from_documents(
    information, storage_context=storage_context, service_context=service_context
)

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [51]:
# load from disk
db2 = chromadb.PersistentClient(path=PATH)
chroma_collection = db2.get_or_create_collection(COLLECTION)

vector_store = ChromaVectorStore(chroma_collection=chroma_collection)

index2 = VectorStoreIndex.from_vector_store(
    vector_store,
    service_context=service_context,
)

In [52]:
query_engine = index2.as_query_engine()

In [53]:
resp = query_engine.query("What is the turing test?")
print(resp.response)

The Turing test is a measure of a machine's ability to exhibit intelligent behavior that is indistinguishable from that of a human. It was proposed by Alan Turing in 1950 as a way to determine whether a machine is truly intelligent or just mimicking intelligence.

The test involves a human evaluator who interacts with both a human and a machine, without knowing which is which. The evaluator is asked to judge the machine's responses to a series of questions or tasks, and to decide whether they appear to be coming from a human or a machine. If the evaluator cannot reliably tell the difference between the human and the machine, then the machine is said to have passed the Turing test.

The Turing test is important because it helps to define the limits of artificial intelligence (AI) and to determine when a machine can be considered truly intelligent. It also raises questions about the nature of intelligence and consciousness, and whether these can be replicated in a machine.


In [54]:
resp = query_engine.query("Generate 2 concise questions about rational agents")

In [55]:
resp.response.split('\n')

['Based on the context information provided, here are two concise questions that could be asked about rational agents:',
 '',
 '1. How do rational agents decide which actions to take in a given situation, and how do they evaluate the potential outcomes of those actions?',
 '2. In what ways can rational agents use heuristics and utility functions to make decisions more efficiently and effectively?']

In [56]:
resp = query_engine.query("When is the final project due")

In [57]:
print(resp.response)

The final project is due on Nov 30, 2023 (tentative).


## Evaluation

We have been quite successful at getting what looks like accurate results, but lets run some evaluations on the generated text. Inspired from the RAGAS paper (https://arxiv.org/pdf/2309.15217v1.pdf), there is a ragas python library which runs evaluations on the RAG generated text. 

**Retrieval Augmentation Generated Assessment** evaluates a RAG generated text without human interference. Even after utilizing RAG we wouldn't be sure if the system hallucinated the generation. The paper proposes a suite of metrics that evaluates RAG.

*Faithfulness*: Measures the factual consistency of the generated answer. It is based on the answer and the retrieved context. The generated answer is faithful if all the answers can be deduced from the given context. The score ranges from 0 to 1. Higher the score, the better. 

*Answer Relevance*: Measures how relevant the answer is to the given prompt. Incomplete and/or redudant answers are given a lower score. The score ranges from 0 to 1.   

*Context Relevance*: Similar to answer relevancy, context relevancy measures the relevance of the generated text to the context. The idea is that the context should exlusively contain the information required to answer the query prompt. 

*Context Precision*: Context Precision is a metric that evaluates whether all of the ground-truth relevant items present in the contexts are ranked higher or not. Ideally all the relevant chunks must appear at the top ranks. This metric is computed using the question and the contexts, with values ranging between 0 and 1, where higher scores indicate better precision.

*Context Recall*: Context recall measures the extent to which the retrieved context aligns with the annotated answer, treated as the ground truth. It is computed based on the ground truth and the retrieved context, and the values range between 0 and 1, with higher values indicating better performance.

Definitions are sourced from https://github.com/explodinggradients/ragas/tree/main/docs/concepts/metrics and the RAGAS paper. 

In [59]:
from ragas.metrics import (
    faithfulness,
    answer_relevancy,
    context_precision,
    context_recall,
)
from ragas.metrics.critique import harmfulness

metrics = [
    faithfulness,
    answer_relevancy,
    context_precision,
    context_recall,
    harmfulness,
]

In [60]:
from ragas.llama_index import evaluate

In [61]:
eval_questions = ["What is the turing test?", 
                  "What is the rational agent approach?"]

eval_answers = ["The Turing test, originally called the imitation game by Alan Turing in 1950,[2] is a test of a machine's ability to exhibit intelligent behaviour equivalent to, or indistinguishable from, that of a human. Turing proposed that a human evaluator would judge natural language conversations between a human and a machine designed to generate human-like responses.",
                ""]

eval_answers = [[a] for a in eval_answers]

In [81]:
result = evaluate(query_engine, metrics, eval_questions, eval_answers) #Takes long to run. 5-10min depending on the number of questions and answers

evaluating with [faithfulness]


100%|██████████| 1/1 [03:24<00:00, 204.46s/it]


evaluating with [answer_relevancy]


  0%|          | 0/1 [00:00<?, ?it/s]huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
100%|██████████| 1/1 [00:02<00:00,  2.76s/it]


evaluating with [context_precision]


  0%|          | 0/1 [00:00<?, ?it/s]huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
100%|██████████| 1/1 [00:08<00:00,  8.09s/it]


evaluating with [context_recall]


  0%|          | 0/1 [00:00<?, ?it/s]huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
100%|██████████| 1/1 [00:37<00:00, 37.85s/it]


evaluating with [harmfulness]


  0%|          | 0/1 [00:00<?, ?it/s]huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
100%|██████████| 1/1 [00:34<00:00, 34.74s/it]


In [83]:
result.to_pandas()

,question,contexts,answer,ground_truths,faithfulness,answer_relevancy,context_precision,context_recall,harmfulness
0,What is the turing test?,[6 Chapter 1.Introduction\ntheso-called totalT...,The Turing Test is a theoretical framework for...,"[The Turing test, originally called the imitat...",0.285714,0.920805,0.0,1.0,0
